In [1]:
import urllib.request
import re
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import json
import requests
import json

def strip_price(price):
    rtn = ''
    aGroup = '0123456789.'
    for i in price:
        if i in aGroup:
            rtn += i
    return rtn

def get_image(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    img = soup.find_all('div', class_='lSSlideOuter')
    for i in img:
        return i.find('li', {'data-src': True})['data-src']


# returns 2 list of http links separated by availability
def luckyGunner_availability(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    page_body = soup.body
    products = soup.find_all('div', class_='main')

    
    # list of all handgun ammo seperated by availability
    products = soup.find_all('div', class_='block-content block-no-padding')
    bulletsList = str(products).split('<div class="main-category-cols">')
    inStock = bulletsList[1]
    outofStock = bulletsList[2]

    inStockScrapedLinks, outofStockScrapedLinks = [], []
    
    inStock = inStock.split("</li>")
    inStockScrapedLinks = [re.findall("(?P<url>https?://[^\s]+)", str(inStockItems)) for inStockItems in inStock]



    outofStock = outofStock.split("</li>")
    outofStockScrapedLinks = [re.findall("(?P<url>https?://[^\s]+)", str(outOfStockItems)) for outOfStockItems in outofStock]

    inStockCleanLinks = []
    outofStockCleanLinks = []
    

    for i in inStockScrapedLinks:
        cleanLink = ""
        for j in i:
            for k in j:
                if k != '"':
                    cleanLink += k
                    
                else:
                    inStockCleanLinks.append(cleanLink)
                    break
                
    for i in outofStockScrapedLinks:
        cleanLink = ""
        for j in i:
            for k in j:
                if k != '"':
                    cleanLink += k
                else:
                    outofStockCleanLinks.append(cleanLink)
                    break
    return inStockCleanLinks, outofStockCleanLinks

In [2]:
def scrapeRifleAmmo(link):
    link+="?limit=all"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    
        
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "lxml")
    page_body = soup.body
    detailPageLinks = []
    links = soup.find_all('h3', class_='product-name')
    detailProduct = soup.find_all('div', class_='product-shop')

    dictLinks = []
    imageURL = []
    for i in links:
        x = i.find('a')
        detailPageLinks = x['href']
        dictLinks.append(detailPageLinks)
        imageURL.append(get_image(detailPageLinks))
    
    names = []
    prices = []
    calibers = []
    try:
        calibers.append(soup.find('div', class_='category-title').text)
    except:
        print(link)

    linkIndex = 0 
    imageIndex = 0
    list_of_dicts = []
    for product in detailProduct:
        aDict = {'name': names, 'prices': prices, 'links': dictLinks, 'caliber': calibers, 'imageUrl': imageURL}

        # scrape names
        name = product.find('a')
        name = name.find('span').text
        aDict['name'] = name
        aDict['imageUrl'] = imageURL[imageIndex]
        

        # scrape prices
        price = product.find('div', class_='price-box')
        if "Special Price" in str(price):
            price = price.find('span', class_='price')
            aDict['prices'] = strip_price(price.text)
        else:
            if 'As low as' in str(price):
                price = (price.find('span', class_='price'))
                aDict['prices'] = strip_price(price.text)
            else:
                try:
                    prices.append(price.text)
                    aDict['prices'] = strip_price(price.text)
                except:
                    prices.append("NOPE")
                    aDict['prices'] = "out of stock"

        aDict['links'] = dictLinks[linkIndex]
        linkIndex += 1
        try:
            aDict['caliber'] = calibers[0].replace('\n', '')
        except:
            print(link)

        list_of_dicts.append(aDict)
    return list_of_dicts
        
    
    


In [3]:
link = "https://www.luckygunner.com/rifle"
x = luckyGunner_availability(link)
ammo_data = {"type":"rifle", "ammo_results": []}
ld = []
for i in x[0]:
    dicts = scrapeRifleAmmo(i)
    for j in dicts:
        ld.append(j)
for i in ld:
    ammo_data['ammo_results'].append(i)



In [ ]:
from pathlib import Path
import os, shutil, glob
rel = "jsonFiles/luckyGunner/ammo"
cwd = Path.cwd()
currentPath = str(Path.cwd())+""
aCount = 0
savePath = str(cwd).replace("pyScripts/luckyGunner", rel)

with open(os.path.join(savePath, f"LGRifleAmmo.json"), "w") as fp:
    json.dump(ammo_data, fp)

rel = 'crispy-computing-app/crispy-computing-app/jsonFiles/luckyGunner/ammo'
cwd = Path.cwd()
currentPath = str(Path.cwd())+''
savePath = str(cwd).replace("pyScripts/luckyGunner", rel)
index = 0
with open(os.path.join(savePath, f"LGRifleAmmo.json"), "w") as fp:
    json.dump(ammo_data, fp)